<div align=center><h1>COMP10001 2019 S2: Foundation of Computing<br>Project 2 Toy World</h1></div>

## Introduction
We often use computers to discover efficient solutions to problems. Artificial intelligence provides a range of techniques for solving a wide variety of problems. In this project, you will implement two techniques for solving a search problem.

Often when developing and demonstrating AI techniques, we use "toy worlds": simplified problems for which a concise, exact definition is possible. This is in contrast to "real world" problems, which are often much messier.

The "toy world" scenario that we will use for this project is as follows:

**Bring me the treasure!**

Falca (a lesser known cousin of Frodo) has ventured into a cave to collect the treasure that lies therein. This cave can be represented as a two-dimensional grid, completely enclosed by stone walls. Falca arrives at the entrance to the cave, and must locate all of the treasure it contains (a known quantity) before departing from the cave exit (a separate location to the entrance).

Unfortunately, the cave is occupied by a dragon, who will not allow Falca to pass unless armed with a sword. Falca hasn't come prepared with a sword, but there is usually one to be found somewhere in the cave.

We want to help Falca discover the most efficient route from entrance to exit that collects all the treasure along the way.

Note that this is similar to the type of problem a navigation application might need to solve in order to provide efficient directions to a destination, or that a delivery company might need to solve in order to deliver parcels efficiently. Hopefully, it is also a little bit entertaining!

The overall problem to solve is to identify the shortest path through an arbitrary cave that will enable Falca to collect all of the treasure and escape from the exit.

We will break this down into several tasks.
- Creating a representation of the cave
- Checking whether a given path is a valid solution to the problem
- Identifying the shortest path between two points in the cave
- Identifying the optimal (best) path that solves the whole problem

## Question 1: Creating a representation of the cave
A valid cave satisfies the following requirements:
- There will only ever be (at most) a single feature (entrance, exit, wall, sword, dragon or treasure) in any given location in the cave.
- There will always be a single entrance and a single exit in the cave.
- There may be (at most) a single sword in the cave.
- There may be (at most) a single dragon in the cave.
- There may be (at most) three treasures in a cave.
- There may be multiple walls in a cave.
- A dragon, if one is present, will never be located in one of the locations (up to eight) adjacent to the entrance.

Note that a valid cave may contain no sword, dragon, treasures and/or walls.

A cave is specified via a dictionary with the following structure:

```python
data = {
  'size': 4,  # the size of the cave
  'entrance': (0, 0),  # the location of the entrance (a row-column tuple)
  'exit': (2, 1),  # the location of the exit
  'dragon': (0, 2),  # the location of the dragon
  'sword': (3, 3),  # the location of the sword
  'treasure': [(1, 3)],  # a list of treasure locations
  'walls': [(1, 1), (1, 2), (2, 2), (2, 3)]  # a list of wall locations
}
```

The different types of location in the cave are identified with different symbols.
- `empty` is denoted by '.';
- `wall` is denoted by '#';
- `entrance` is denoted by '@';
- `exit` is denoted by 'X';
- `treasure` is denoted by a '$';
- `sword` is denoted by 't'; and
- `dragon` is denoted by 'W'.

The cave corresponding to the specification above is shown in Figure 1.

![Example Cave](../img/Figure_1.png)

Your first task is to write a function build_cave(data) that builds a two-dimensional representation of a cave from a data dictionary specifying the features of that cave as described on the slide before. The function takes a single argument data, a dictionary of features in the cave, and returns a representation of the cave as a two-dimensional array (ie, a list of lists), or None if the dictionary of features specifies a non-valid cave, according to the rules on the previous slide.

In [90]:
chars = {
        'empty': '.',
        'entrance': '@',
        'exit': 'X',
        'wall': '#',
        'sword': 't',
        'dragon': 'W',
        'treasure': '$',
        }

def wset(cave, loc, feature):
    '''Shortcut function for setting a feature at a cave location.'''
    i, j = loc
    cave[i][j] = feature

def wget(cave, loc):
    '''Shortcut function for getting a feature at a cave location.'''
    i, j = loc
    return cave[i][j]

def get_neighbours(loc, size, diag=False):
    '''Shortcut function for getting a list of valid neighbours of a 
    location. If diag == True, include diagonal neighbours'''
    i, j = loc
    candidates = [(i + 1, j), (i - 1, j), (i, j + 1), (i, j - 1)]
    if diag:
        candidates.extend([(i + 1, j + 1), (i + 1, j - 1), 
                           (i - 1, j + 1), (i - 1, j - 1)])
    return [x for x in candidates if 0 <= x[0] < size and 0 <= x[1] < size]

def check_valid(cave, size, loc):
    '''Shortcut function for checking that a location is (i) located 
    within the cave, and (ii) currently empty.'''
    i, j = loc
    return 0 <= i < size \
            and 0 <= j < size \
            and wget(cave, loc) == chars['empty']

def build_cave(data):
    '''The main function to build and validate a cave specified by the
    provided data dictionary. Returns a 2D array describing the cave 
    layout if the specification is valid, or None otherwise.'''

    # build an empty cave
    if 'size' not in data:
        return None
    cave = [[chars['empty']] * data['size'] for i in range(data['size'])]

    # check that entrance and exit both exist
    if 'entrance' not in data or 'exit' not in data:
        return None
    # check that entrance is valid and add to cave
    if not check_valid(cave, data['size'], data['entrance']):
        return None
    wset(cave, data['entrance'], chars['entrance'])
    # check that exit is valid and add to cave
    if not check_valid(cave, data['size'], data['exit']):
        return None
    wset(cave, data['exit'], chars['exit'])
    # if a sword exists, check that it is valid and add to cave
    if 'sword' in data:
        if not check_valid(cave, data['size'], data['sword']):
            return None
        wset(cave, data['sword'], chars['sword'])
    # if a dragon exists, check that it is valid and add to cave
    if 'dragon' in data:
        if not check_valid(cave, data['size'], data['dragon']):
            return None
        # also need to check that it is not adjacent to entrance
        if data['dragon'] in get_neighbours(data['entrance'], data['size'], 
                                            diag=True):
            return None
        wset(cave, data['dragon'], chars['dragon'])
    # if there is any treasure, check locations are valid and add to cave
    for cur_treasure in data.get('treasure', []):
        if not check_valid(cave, data['size'], cur_treasure):
            return None
        wset(cave, cur_treasure, chars['treasure'])
    # if there are any walls, check locations are valid and add to cave
    for cur_wall in data.get('walls', []):
        if not check_valid(cave, data['size'], cur_wall):
            return None
        wset(cave, cur_wall, chars['wall'])
    # if we have made it here, specification is valid
    return cave

In [91]:
data = {'size': 4,
        'entrance': (0, 0),
        'exit': (2, 1),
        'dragon': (0, 2),
        'sword': (3, 3),
        'treasure': [(1, 3)],
        'walls': [(1, 1), (1, 2), (2, 2), (2, 3)]}

data1 = {'size': 4,
        'entrance': (0, 0),
        'exit': (3, 4),
        'dragon': (0, 2),
        'sword': (3, 3),
        'walls': [(1, 3)],
        'treasure': [(1, 1), (1, 2), (2, 2), (2, 3)]}

In [92]:
build_cave(data)

[['@', '.', 'W', '.'],
 ['.', '#', '#', '$'],
 ['.', 'X', '#', '#'],
 ['.', '.', '.', 't']]

## Question 2: Checking whether a given path is a valid solution to the problem

Your next task is to determine whether a given path is a valid solution to the problem. Note that, at this stage, you don't need to determine whether the path is shortest possible path, simply that it doesn't break any of the following rules:

1. Falca begins at the entrance.
2. Falca can only move one location at a time; eg, from (0, 0) to (1, 0).
3. Falca can only move North (`up`), South (`down`), West (`left`) or East (`right`); ie, diagonal moves are not allowed.
4. Falca can never move off the edge of the size-by-size grid constituting the cave.
5. Falca can never move into a location containing a cave wall (`#`).
6. Falca cannot move into a location containing a dragon (`W`) or any of the eight locations adjacent to a dragon (ie, including diagonally adjacent) unless carrying the sword (`t`), in which case it is considered equivalent to an empty location.
7. Falca must end at the exit (`X`), after having collected all of the the treasures (`$`) in the cave. Until all of the treasures have been collected, the exit location is equivalent to an empty location (`.`); ie, Falca may freely move into and out of it.
8. For Question 2, any item (treasure or sword) in the location currently occupied by Falca is considered to be collected, and no items are ever dropped.
9. Note that if the sword is in one of the (up to) eight locations adjacent to a dragon, then it cannot be collected, as Falca may not move into a location adjacent to a dragon unless already carrying the sword!

![Valid Path](../img/Figure_2.png)

You need to implement a function check_path(data, path) that takes two arguments data, a dictionary of features in the cave, as per Question 1, and a list of moves path constituting a path through the dungeon. Your function should return True if the path is valid, or False if it is not valid. You may assume that the cave specification is valid (ie, build_cave(data) will not return None). However, it is possible that even a valid cave specification will have no valid path through it; for example, if the exit is completely separated from the entrance by walls. When there is no possible valid path through the cave, check_path will return False for any path.

In [93]:
chars = {
        'empty': '.',
        'entrance': '@',
        'exit': 'X',
        'wall': '#',
        'sword': 't',
        'dragon': 'W',
        'treasure': '$',
        }

def wget(cave, loc):
    i, j = loc
    '''Shortcut function for getting a feature at a cave location.'''
    return cave[i][j]

def get_neighbours(loc, size, diag=False):
    '''Shortcut function for getting a list of valid neighbours of a 
    location. If diag == True, include diagonal neighbours'''
    i, j = loc
    candidates = [(i + 1, j), (i - 1, j), (i, j + 1), (i, j - 1)]
    if diag:
        candidates.extend([(i + 1, j + 1), (i + 1, j - 1), 
                           (i - 1, j + 1), (i - 1, j - 1)])
    return [x for x in candidates if 0 <= x[0] < size and 0 <= x[1] < size]

def get_next_loc(cur_loc, move, size):
    '''Shortcut function to obtain valid next location given a move 
    direction. Returns None if next location is invalid.'''
    if move == 'N': 
        next_loc = (cur_loc[0] - 1, cur_loc[1])
    if move == 'E': 
        next_loc = (cur_loc[0], cur_loc[1] + 1)
    if move == 'S': 
        next_loc = (cur_loc[0] + 1, cur_loc[1])
    if move == 'W': 
        next_loc = (cur_loc[0], cur_loc[1] - 1)
    if 0 <= next_loc[0] < size and 0 <= next_loc[1] < size:
        return next_loc
    else:
        return None

def check_path(data, path):
    '''Return True if path is a valid solution to the cave defined in data,
    False otherwise.'''
    # build the cave and count treasures
    cave = build_cave(data)
    max_treasure = len(data['treasure'])

    # treasures found (a set, so we won't count the same treasure twice)
    treasures = set()
    has_sword = False
    cur_loc = data['entrance']
    
    # evaluate each move in turn
    while len(path):
        next_move = path.pop(0)
        # get the next location
        next_loc = get_next_loc(cur_loc, next_move, data['size'])
        # check if it (i) exists, and (b) is passable
        if not next_loc:
            return False
        if wget(cave, next_loc) == chars['wall']:
            return False
        # if moving near/through a dragon, must have a sword
        if not has_sword and (
            wget(cave, next_loc) in [chars['dragon']] or 
            ('dragon' in data and data['dragon'] in 
             get_neighbours(next_loc, data['size'], diag=True))):
            return False
        if wget(cave, next_loc) == chars['sword']:
            has_sword = True
        if wget(cave, next_loc) == chars['treasure']:
            treasures.add(next_loc)
        # last move should be on to exit, with all treasures
        if not path and wget(cave, next_loc) == chars['exit'] \
                and len(treasures) == max_treasure:
            return True
        cur_loc = next_loc

    # if we haven't returned True, the path is invalid.
    return False

In [94]:
data = {'size': 4,'entrance': (0, 0), 'exit': (2, 1),
        'dragon': (0, 2),'sword': (3, 3),'treasure': [(1, 3)],
        'walls': [(1, 1), (1, 2), (2, 2), (2, 3)]}

path = (['S'] * 3 + ['E'] * 3 + ['W'] * 3 + ['N'] * 3 + ['E'] * 3 + ['S'] + ['N'] + ['W'] * 3 + ['S', 'S', 'E'])

In [95]:
check_path(data, path)

True

## Question 3: Identifying the shortest path between two points in the cave

Your next problem is to identify the length of the shortest valid path between two locations in a cave. The solution will be the minimum number of valid moves required to move form the start location to the end location. We will formulate this problem as a search problem and solve it using an algorithm known as breadth-first search.

A search process is often visualised as a tree structure: we begin at the root node of our tree, corresponding to the the initial state of the problem, with Falca at the start location. We then add branches corresponding to each of the possible moves from the entrance. Each of these child nodes then corresponds to the new state of the problem after that move has been executed. We are trying to identify a path to the node corresponding to the goal of the problem, with Falca at the end location.

For example, if our root node corresponds to the state of Falca just having entered the cave at (0, 0), there are only two possible moves that can be executed: South or East, resulting in new child nodes corresponding to the state of Falca having moved to (1, 0) or (0, 1) respectively. The next set of child nodes can then be created based on the valid moves available from that location.

![Valid Path](../img/Figure_3a.png)

At some point in the tree, we will create a node corresponding to the state of Falca having reached our end location. The sequence of moves (ie, branches in our tree) required to get from the initial root node to this goal node is our solution.

An example breadth-first search is illustrated in the figures below.

![Valid Path](../img/Figure_3b.png)

![Valid Path](../img/Figure_3c.png)

![Valid Path](../img/Figure_3d_new.png)

![Valid Path](../img/Figure_3e_new.png)

Breadth-first search is a simple strategy for solving such search problems. Put simply, in breadth-first search, we first create all of the child nodes of our root node (start location), then all of the children of those nodes, and so on. We always explore all of the possible states that are n moves from the root node, before exploring states that are n + 1 moves from the root node. This guarantees that the first time we reach the goal node, no shorter path will exist. (If it did, we would have already found it.)

A simple way of implementing breadth-first search is to add nodes to be explored to a queue. A queue is simply a list where we add elements to the back and remove elements from the front. Much the same as we join a queue to board a bus at the back of the queue, and board the bus once we reach the front of the queue.

Pseudocode for breadth-first search is provided below:

```
node = start location
unexplored = a queue containing node as the only element
explored = an empty set

repeat:
    if unexplored is empty: 
        return failure (there is no path)
    node = node at front of unexplored
    if node == end location: 
        return solution
    add node to explored
    for each possible move from node:
        child = result of executing move
        if child is not in either unexplored or explored:
            add child to back of unexplored 
```

This pseudocode can be to solve the question.

Write a function shortest_path(data, start, end, has_sword) that determines the length of the shortest valid path between two locations in a cave. The function takes four arguments data a dictionary of features in the cave, as per question 1; start a start location (as a tuple); end an end location (as a tuple); and has_sword a boolean indicating whether Falca currently holds the sword (as this may affect the choice of path). Your function should return the length of the shortest path between start and end.

For simplicity, you should assume that the value of has_sword does not change while the shortest path is determined. That is, Falca cannot pick up the sword; If shortest_path is called with the argument False for has_sword, it will remain False, even if Falca happens to enter a location containing the sword.

In [97]:
chars = {
        'empty': '.',
        'entrance': '@',
        'exit': 'X',
        'wall': '#',
        'sword': 't',
        'dragon': 'W',
        'treasure': '$',
        }

def wget(cave, loc):
    '''Shortcut function for getting a feature at a cave location.'''
    i, j = loc
    return cave[i][j]

def get_neighbours(loc, size, diag=False):
    '''Shortcut function for getting a list of valid neighbours of a 
    location. If diag == True, include diagonal neighbours'''
    i, j = loc
    candidates = [(i + 1, j), (i - 1, j), (i, j + 1), (i, j - 1)]
    if diag:
        candidates.extend([(i + 1, j + 1), (i + 1, j - 1), 
                           (i - 1, j + 1), (i - 1, j - 1)])
    return [x for x in candidates if 0 <= x[0] < size and 0 <= x[1] < size]

def shortest_path(data, start, end, has_sword):
    '''Return the shortest path from start to end in the cave defined in
    data, taking into account whether a sword is currently held.'''
    cave = build_cave(data)

    # queue of locations to be explored
    queue = [(start, [])]
    # a set of previously visited locations
    visited = set()

    # while there are still locations to be explored
    while queue:
        # get the location at the head of the queue
        (cur_state, cur_path) = queue.pop(0)
        visited.add(cur_state)

        # we've reached the end; return the path length
        if cur_state == end:
            return len(cur_path)

        # otherwise, consider all (valid) neighbouring locations in turn
        for next_state in get_neighbours(cur_state, data['size']):
            # don't revisit an already visited location
            if next_state in visited:
                continue
            # don't add a location to the queue that already exists
            if next_state in queue:
                continue
            # can't move through walls
            if wget(cave, next_state) == chars['wall']:
                continue
            # can't move through dragon unless carrying sword
            if not has_sword and (
                wget(cave, next_state) in [chars['dragon']] or 
                ('dragon' in data and data['dragon'] in 
                 get_neighbours(next_state, data['size'], diag=True))):
                continue
            # otherwise, location is a valid move, add to queue
            queue.append((next_state, cur_path + [next_state]))

    # if queue is empty and we haven't reached end, then no valid path exists
    return None 

In [98]:
data = {'size': 4,'entrance': (0, 0),'exit': (2, 1),'dragon': (0, 2),'sword': (3, 3),'treasure': [(1, 3)],'walls': [(1, 1), (1, 2), (2, 2), (2, 3)]}
print(shortest_path (data, data['entrance'], data['sword'], False))
print(shortest_path (data, data['entrance'], data['treasure'][0], False))

6
None


## Question 4: Identifying the optimal (best) path that solves the whole problem

Your final task is to write a function that determines the length of the optimal path from entrance to exit, collecting all of the treasure.

In Question 3, we used breadth-first search to identify the shortest path between two locations in a cave. This provides us with the means to quantify the cost of travelling between any pair of locations. In this question, we will increase the scope of the problem to search for the optimal order in which to collect each of the treasures and then exit from a cave, optionally also collecting the sword, but only if that will reduce the overall path length.

To solve this problem, we can now consider our moves to be from one location of interest to another; e.g., from the entrance to the first treasure, or from the sword to the exit, and so on. Each of these movements can be associated with a cost, defined as the length of the path from the start location to the end location, as calculated by the shortest_path function from Question 3. Note that, as the definition of shortest_path specified that the sword could not be collected during a path between a start and end location, the sword will only be considered collected if it is the end location of a call to shortest_path.

We can solve this problem using a variant of breadth-first search known as uniform cost search. Breadth-first search finds the optimal solution because the cost of executing each move was equal. When this is not true, we need to keep track of not only the number of moves we have made, but also the cumulative cost of making those moves. Rather than exploring new nodes that are the fewest number of moves from the root node, we now want to explore new nodes that are the lowest cost (path length) from the root node.

Thus, rather than storing unexplored nodes as a simple queue, we will store unexplored nodes as a priority queue in which we order unexplored nodes according to the cost of reaching them. At each decision point, we choose to explore the next cheapest node; that is, the one with the lowest cost, or the shortest path length from the root node.

Pseudocode for a uniform-cost search is provided below:

```
node = start state
unexplored = a priority queue containing node as the only element

repeat:
    if unexplored is empty: 
        return failure (there is no valid path)
    node = node in unexplored with the lowest cost
    if node == end state:
        return solution
    for each valid move from node:
        child = result of executing move
        if child is not in unexplored:
            add child to unexplored, together with its cost
        else if child is in unexplored and the new cost is less than 
                    current cost:
            replace the current cost with new cost
```

You need to implement a function optimal_path(data) that takes a single argument data, a dictionary of features in the cave, as per Question 1. Your function should return the length of the shortest path that enables Falca to collect all of the treasures and exit from the dungeon.

In [100]:
def optimal_path(data):
    '''Return the length of the optimal path that 'solves' the cave defined by
    data; ie, collects all treasures and reaches the exit.'''
    # locations of interest
    locations = [data['exit']]
    if 'sword' in data:
        locations.append(data['sword'])
    if 'treasure' in data:
        locations += data['treasure']

    # state = (current_location, visited_locations, has_sword, treasure_count)
    initial_state = (data['entrance'], (), False, 0)

    # total number of treasures to collect
    treasure_max = len(data['treasure']) if 'treasure' in data else 0

    # queue: here a dictionary mapping state to path cost
    queue = {initial_state: 0}
    
    # while there are still states to be explored
    while queue:
        # get the lowest cost state, remove from the queue and unpack
        cur_state = min(queue, key=queue.get)
        cur_cost = queue.pop(cur_state)
        cur_loc, visited, has_sword, treasure_count = cur_state

        # if we've satisfied the goal conditions (all treasure collected 
        # and at exit), return
        if cur_loc == data['exit'] and treasure_count == treasure_max:
            return cur_cost

        # otherwise, explore next locations
        for next_loc in locations:
            # don't revisit an already visited location
            if next_loc in visited:
                continue
            # generate the path to the next location
            cost = shortest_path(data, cur_loc, next_loc, has_sword)
            # continue if no path to the next location
            if cost is None:
                continue
            # otherwise, generate the next state, and add it to the queue 
            # if either: (i) it doesn't yet exist; or (ii) it exists, but 
            # this is a shorter path
            next_state = (next_loc, visited + (next_loc,), 
                    has_sword or 
                          ('sword' in data and next_loc == data['sword']), 
                    treasure_count + (next_loc in data['treasure']
                        if 'treasure' in data else 0))
            next_cost = cur_cost + cost
            if next_state not in queue or queue[next_state] > next_cost:
                queue[next_state] = next_cost

    # if queue is empty and we haven't reached goal state
    return None

In [101]:
data = {'size': 4,'entrance': (0, 0),'exit': (2, 1),'dragon': (0, 2),'sword': (3, 3),'treasure': [(1, 3)],'walls': [(1, 1), (1, 2), (2, 2), (2, 3)]}
print(optimal_path(data))

data = {'size': 5,'entrance': (0, 2),'exit': (0, 4),'sword': (0, 0),'dragon': (2, 1),'treasure': [(0, 3), (4, 1)],'walls': [(1, 0), (2, 0), (3, 0), (1, 2), (2, 2), (3, 2), (1, 3), (2, 3), (3, 3)]}
print(optimal_path(data))

23
14
